# EN einhliðamálheild
Í þessu reikniriti eru enskar einhliðamálheildir lesnar og forunnar svo ein setning sé í hverri línu. Málheildirnar eru eftirfrandi.

| Nafn | #línur| stærð | ath |
| --- | --- | --- | --- |
| newscrawl 2018 en | 18,1M | 2 GB | - |
| UN corpus EN | 27,4M | 3,4GB | - |
| Europarl v9 | 2,3M | 321 MB | - |
| Librispeech 27 | 48,2M | 4,3GB | - | - |
| Wikipedia Google en | 25,8M | 3,0GB | - |
| --- | --- | --- | --- |
| Heild | 121,9M | 13,1GB | - |

In [1]:
import os
import pathlib
from pathlib import Path
import re
from pprint import pprint

import frontend.core as c
import frontend.bulk as b

%matplotlib notebook
c.THREADS = 10
c.MEMORY = 17179869184 # 16GB

working_dir = pathlib.Path('/work/haukurpj')
data_dir = working_dir.joinpath('data')
processing_dir = working_dir.joinpath('process')
p = processing_dir
en_mono = data_dir.joinpath('en-monolingual')

IS = c.Lang.IS
EN = c.Lang.EN

NEWS, UN, EUROPARL, LIBRI, WIKI = 'news', 'un', 'europarl', 'libri', 'wiki'

langs = [IS, EN]

[nltk_data] Downloading package punkt to
[nltk_data]     /home/staff/haukurpj/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2019-12-13 14:07:16,649 - frontend - INFO - Initialized


## Europarl EN
Þessi gögn líta vel út og ekkert þarf að gera.

In [10]:
euro_parl = en_mono.joinpath('europarl-v9.en')
print(b.info_formatted(euro_parl))
print(*b.peek(euro_parl))

/work/haukurpj/data/en-monolingual/europarl-v9.en   321.2MiB       2295044
Resumption of the session
 I declare resumed the session of the European Parliament adjourned on Friday, 15 December 2000.
 Statements by the President
 Ladies and gentlemen, on Saturday, as you know, an earthquake struck Central America once again, with tragic consequences.
 This is an area which has already been seriously affected on a number of occasions since the beginning of the twentieth century.
 The latest, provisional, figures for victims in El Salvador are already very high.
 There are 350 people dead, 1 200 people missing, the area is completely devastated and thousands of homes have been destroyed throughout the country.
 The European Union has already shown its solidarity by sending a rescue team to the area, whilst financial assistance from the Union and Member States has been, or is in the process of being, released and I am able to inform you that some groups in the European Parliament have reque

In [12]:
!cp {euro_parl} {b.write(p, EN, EUROPARL)}

## News crawl 2018
Lítur vel út - ekkert sem þarf að gera.

In [13]:
news = en_mono.joinpath('news.2018.en.shuffled.deduped')
print(b.info_formatted(news))
print(*b.peek(news))

/work/haukurpj/data/en-monolingual/news.2018.en.shuffled.deduped    2.0GiB       18113311
He owns the Orphaned Wildlife Center with his wife, Susan.
 Qatar also made a loose commitment not to launch flights to the United States from Europe or other non-Qatari cities that would create yet more competition for U.S. carriers.
 The fact there's a banker pig as well is kind of amusing.
 Berlinger was the "gold standard" and "a huge hole that we have to fill."
 "From tonight on, I'm sorry for them, but they are going to be different," coach Didier Deschamps said.
 :: The top 10 beaches in the UK according to TripAdvisor are:
 Morgan, who has a lengthy criminal history including drugs offences, had not been investigated for suspected terrorism before.
 But despite the major disruption caused and resulting financial bill, Judge Philip Statman said Khonde had already been sufficiently punished by being on remand since his arrest and serving the equivalent of 14 months' custody.
 Just type Bono 

In [14]:
!cp {news} {b.write(p, EN, NEWS)}

## Wikipedia
Þessi gögn eru í `.csv`, við þurfum að laga það.

In [27]:
import pandas as pd
wiki = en_mono.joinpath('documents_utf8_filtered_20pageviews.csv')
wiki_df = pd.read_csv(str(wiki), sep=",", header=None)
wiki_df.head()

,0,1
0,wikipedia-23885690,Research Design and Standards Organization T...
1,wikipedia-23885928,The Death of Bunny Munro The Death of Bunny ...
2,wikipedia-23886057,Management of prostate cancer Treatment for ...
3,wikipedia-23886425,Cheetah reintroduction in India Reintroducti...
4,wikipedia-23886491,Langtang National Park The Langtang National...


In [28]:
wiki_df = wiki_df[1]
wiki_df.head()

0     Research Design and Standards Organization  T...
1     The Death of Bunny Munro  The Death of Bunny ...
2     Management of prostate cancer  Treatment for ...
3     Cheetah reintroduction in India  Reintroducti...
4     Langtang National Park  The Langtang National...
Name: 1, dtype: object

Vistum þau í réttri skrá.

In [29]:
wiki_df.to_csv(b.write(p, EN, WIKI), header=None, index=None, sep=' ', mode='w')

/home/staff/haukurpj/.conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [30]:
print(b.info_formatted(b.read(p, EN, WIKI)))
print(*b.peek(b.read(p, EN, WIKI)))

/work/haukurpj/process/wiki.en              3.0GiB         463819
" Research Design and Standards Organization  The Research Design and Standards Organisation (RDSO) is an ISO 9001 research and development organisation under the Ministry of Railways of India, which functions as a technical adviser and consultant to the Railway Board, the Zonal Railways, the Railway Production Units, RITES and IRCON International in respect of design and standardisation of railway equipment and problems related to railway construction, operation and maintenance. History. To enforce standardisation and co-ordination between various railway systems in British India, the Indian Railway Conference Association (IRCA) was set up in 1903. It was followed by the establishment of the Central Standards Office (CSO) in 1930, for preparation of designs, standards and specifications. However, till independence in 1947, most of the designs and manufacture of railway equipments was entrusted to foreign consultants. Af

Núna er skjalið með eina grein í hverri línu, auka gæsalappir eru í kringum allan textann og setningarnar innihalda tvíteknar gæsalappir, lögum það.

In [32]:
import nltk
import re

with b.read(p, EN, WIKI).open() as f_in, b.write(p, EN, WIKI, 'final').open('w+') as f_out:
    for line in f_in:
        # we strip away the " at the end and beginning.
        line = line[2:]
        line = line[:-2]
        line = re.sub(r'""', r'"', line)
        for sent in nltk.sent_tokenize(line):
            f_out.write(sent + "\n")

In [33]:
print(b.info_formatted(b.read(p, EN, WIKI, 'final')))
print(*b.peek(b.read(p, EN, WIKI, 'final')))

/work/haukurpj/process/wiki-final.en        3.0GiB       25775880
Research Design and Standards Organization  The Research Design and Standards Organisation (RDSO) is an ISO 9001 research and development organisation under the Ministry of Railways of India, which functions as a technical adviser and consultant to the Railway Board, the Zonal Railways, the Railway Production Units, RITES and IRCON International in respect of design and standardisation of railway equipment and problems related to railway construction, operation and maintenance.
 History.
 To enforce standardisation and co-ordination between various railway systems in British India, the Indian Railway Conference Association (IRCA) was set up in 1903.
 It was followed by the establishment of the Central Standards Office (CSO) in 1930, for preparation of designs, standards and specifications.
 However, till independence in 1947, most of the designs and manufacture of railway equipments was entrusted to foreign consultants.


## UN Corpus en 
Þessi málheild byggist á mörgum .tei skjölum. Við þurfum að lesa öll skjölin og sameina í eitt stórt skjal. Þar sem flest skjöl byrja á fundarlýsingu og öðrum setningum sem eru ekki sérlega góðar setningar. Við byrjum því að lesa setningar eftir setningu númer 40.

In [34]:
un_dir = en_mono.joinpath('UNv1.0-TEI')
xml_files = !find {un_dir} -name '*.xml'
xml_files = [Path(xml_file) for xml_file in xml_files]

In [35]:
!tail -n 60 {xml_files[0]}

      </p>
      <p id="38">
        <s id="38:1" lang="en">Educational background:</s>
      </p>
      <p id="39">
        <s id="39:1" lang="en">Harvard University, Kennedy School of Government</s>
      </p>
      <p id="40">
        <s id="40:1" lang="en">Mid Career/ Master in Public Administration, 2005</s>
      </p>
      <p id="41">
        <s id="41:1" lang="en">University of Tirana, Albania</s>
      </p>
      <p id="42">
        <s id="42:1" lang="en">Bachelor in Law, 2000; Professor in Sociology, 1999,</s>
      </p>
      <p id="43">
        <s id="43:1" lang="en">PhD in Sociology, 1989; BA in Political Sciences, 1984</s>
      </p>
      <p id="44">
        <s id="44:1" lang="en">Other main activities in the field relevant to the mandate of the treaty body concerned:</s>
      </p>
      <p id="45">
        <s id="45:1" lang="en">Expert on the following issues:</s>
      </p>
      <p id="46">
        <s id="46:1" lang="en">Migration, Ethnicity, Minorities, Human Rights

In [49]:
from xml.etree import ElementTree as ET

def tei_read_file(path: Path):
    sentences = []
    try: 
        root = ET.parse(str(path)).getroot()
        # We gather all the paragraphs from the body, avoiding the divs
        for paragraph_node in root.findall('.//body//p'):
            # we only take sentences which have an "id" high enough.
            if int(paragraph_node.attrib['id']) < 40:
                continue
            for sentence_node in paragraph_node.findall('.//s'):
                sentence = sentence_node.text
                sentences.append(sentence)
    except:
        # We just skip the file
        pass
    return sentences

In [50]:
print(tei_read_file(xml_files[0]))

['Mid Career/ Master in Public Administration, 2005', 'University of Tirana, Albania', 'Bachelor in Law, 2000; Professor in Sociology, 1999,', 'PhD in Sociology, 1989; BA in Political Sciences, 1984', 'Other main activities in the field relevant to the mandate of the treaty body concerned:', 'Expert on the following issues:', 'Migration, Ethnicity, Minorities, Human Rights, Management, Strategic planning, Organizational development, Reengineering, Restructuring, Employment, Social Policies, Social Services, Electoral Systems and Elections, Negotiation/International cooperation, Euro-Atlantic Integration, Project coordination and management.', 'Country Representative:', 'Council of Europe, Committee of Social Charter, 1999-2004; Committee of Social Cohesion, 1999-2001, 2005-present; Committee on Migration, 1998-2002.', 'List of most recent publications in the field:', 'Barjaba, K. 2003 Shqiptaret.', 'Keta ikes te medhenj (Albanians, These Big Leavers), Tirane:Korbi', 'Barjaba, K. 2002 O

In [51]:
with b.write(p, EN, UN).open('w+') as f_out:
    for xml_file in xml_files:
        sentences = tei_read_file(xml_file)
        for sentence in sentences:
            if sentence:
                f_out.write(sentence + '\n')

In [52]:
print(b.info_formatted(b.read(p, EN, UN)))
print(*b.peek(b.read(p, EN, UN), length=100))

/work/haukurpj/process/un.en                3.4GiB       27458880
Mid Career/ Master in Public Administration, 2005
 University of Tirana, Albania
 Bachelor in Law, 2000; Professor in Sociology, 1999,
 PhD in Sociology, 1989; BA in Political Sciences, 1984
 Other main activities in the field relevant to the mandate of the treaty body concerned:
 Expert on the following issues:
 Migration, Ethnicity, Minorities, Human Rights, Management, Strategic planning, Organizational development, Reengineering, Restructuring, Employment, Social Policies, Social Services, Electoral Systems and Elections, Negotiation/International cooperation, Euro-Atlantic Integration, Project coordination and management.
 Country Representative:
 Council of Europe, Committee of Social Charter, 1999-2004; Committee of Social Cohesion, 1999-2001, 2005-present; Committee on Migration, 1998-2002.
 List of most recent publications in the field:
 Barjaba, K. 2003 Shqiptaret.
 Keta ikes te medhenj (Albanians, These Big Le

## Libri Speech 27

Þessi málheild er samansafn af bókum og inniheldur mikið af tómum línum. Einnig er hver bók í sér skrá.

In [53]:
libri_dir = en_mono.joinpath('librispeech').joinpath('corpus')

In [54]:
txt_files = !find {libri_dir} -name '*.txt'
txt_files = [Path(txt_file) for txt_file in txt_files]

In [55]:
txt_files[:10]

[PosixPath('/work/haukurpj/data/en-monolingual/librispeech/corpus/2834/2834.txt'),
 PosixPath('/work/haukurpj/data/en-monolingual/librispeech/corpus/2865/2865.txt'),
 PosixPath('/work/haukurpj/data/en-monolingual/librispeech/corpus/3016/3016.txt'),
 PosixPath('/work/haukurpj/data/en-monolingual/librispeech/corpus/3184/3184.txt'),
 PosixPath('/work/haukurpj/data/en-monolingual/librispeech/corpus/3185/3185.txt'),
 PosixPath('/work/haukurpj/data/en-monolingual/librispeech/corpus/3335/3335.txt'),
 PosixPath('/work/haukurpj/data/en-monolingual/librispeech/corpus/3690/3690.txt'),
 PosixPath('/work/haukurpj/data/en-monolingual/librispeech/corpus/3755/3755.txt'),
 PosixPath('/work/haukurpj/data/en-monolingual/librispeech/corpus/3782/3782.txt'),
 PosixPath('/work/haukurpj/data/en-monolingual/librispeech/corpus/4069/4069.txt')]

In [57]:
!tail -n 10 {txt_files[0]}


On which he looked up at her--but only to guess, from her face, with a
revulsion, that she simply meant he was young. She stood shining at him
with that cheap comfort, and it added, on the spot, thirty years to his
life. She walked him away with her, however, as if she had given him now
the key to patience.






In [61]:
!head -n 50 {txt_files[0]}






THE PORTRAIT OF A LADY

VOLUME II (of II)


By Henry James




CHAPTER XXVIII

On the morrow, in the evening, Lord Warburton went again to see his
friends at their hotel, and at this establishment he learned that they
had gone to the opera. He drove to the opera with the idea of paying
them a visit in their box after the easy Italian fashion; and when
he had obtained his admittance--it was one of the secondary
theatres--looked about the large, bare, ill-lighted house. An act
had just terminated and he was at liberty to pursue his quest. After
scanning two or three tiers of boxes he perceived in one of the largest
of these receptacles a lady whom he easily recognised. Miss Archer was
seated facing the stage and partly screened by the curtain of the box;
and beside her, leaning back in his chair, was Mr. Gilbert Osmond. They
appeared to have the place to themselves, and Warburton supposed their
companions had taken advantage of the recess to enjoy the relative
coolness of the lobby.

In [64]:
import nltk
import re

def read_libri_file(p_in):
    sentences = []
    buffer = []
    with p_in.open() as f_in:
        for line in f_in:
            line = line.strip()
            if not line:
                if buffer:
                    for sent in nltk.sent_tokenize(" ".join(buffer)):
                        sentences.append(sent)
                    buffer = []
            else:
                buffer.append(line)
    if buffer:
        for sent in nltk.sent_tokenize(" ".join(buffer)):
            sentences.append(sent)
        buffer = []
    return sentences

In [69]:
read_libri_file(txt_files[0])[:16]

['THE PORTRAIT OF A LADY',
 'VOLUME II (of II)',
 'By Henry James',
 'CHAPTER XXVIII',
 'On the morrow, in the evening, Lord Warburton went again to see his friends at their hotel, and at this establishment he learned that they had gone to the opera.',
 'He drove to the opera with the idea of paying them a visit in their box after the easy Italian fashion; and when he had obtained his admittance--it was one of the secondary theatres--looked about the large, bare, ill-lighted house.',
 'An act had just terminated and he was at liberty to pursue his quest.',
 'After scanning two or three tiers of boxes he perceived in one of the largest of these receptacles a lady whom he easily recognised.',
 'Miss Archer was seated facing the stage and partly screened by the curtain of the box; and beside her, leaning back in his chair, was Mr. Gilbert Osmond.',
 'They appeared to have the place to themselves, and Warburton supposed their companions had taken advantage of the recess to enjoy the relati

Við náum ekki alveg textanum réttum með NLTK. En við köllum það gott.

In [70]:
with b.write(p, EN, LIBRI).open('w+') as f_out:
    for txt_file in txt_files:
        for sentence in read_libri_file(txt_file):
            f_out.write(sentence + '\n')

In [71]:
print(b.info_formatted(b.read(p, EN, LIBRI)))
print(*b.peek(b.read(p, EN, LIBRI), length=100))

/work/haukurpj/process/libri.en             4.3GiB       48296733
THE PORTRAIT OF A LADY
 VOLUME II (of II)
 By Henry James
 CHAPTER XXVIII
 On the morrow, in the evening, Lord Warburton went again to see his friends at their hotel, and at this establishment he learned that they had gone to the opera.
 He drove to the opera with the idea of paying them a visit in their box after the easy Italian fashion; and when he had obtained his admittance--it was one of the secondary theatres--looked about the large, bare, ill-lighted house.
 An act had just terminated and he was at liberty to pursue his quest.
 After scanning two or three tiers of boxes he perceived in one of the largest of these receptacles a lady whom he easily recognised.
 Miss Archer was seated facing the stage and partly screened by the curtain of the box; and beside her, leaning back in his chair, was Mr. Gilbert Osmond.
 They appeared to have the place to themselves, and Warburton supposed their companions had taken advant

## Sameinum í eitt skjal

In [72]:
corpora = [
    b.read(p, EN, EUROPARL),
    b.read(p, EN, NEWS),
    b.read(p, EN, WIKI, 'final'),
    b.read(p, EN, UN),
    b.read(p, EN, LIBRI)
]

b.combine(corpora, b.write(p, EN, 'mono'))

True

In [73]:
print(b.info_formatted(b.read(p, EN, 'mono')))
print(*b.peek(b.read(p, EN, 'mono'), length=100))

/work/haukurpj/process/mono.en              13.1GiB     121939848
Resumption of the session
 I declare resumed the session of the European Parliament adjourned on Friday, 15 December 2000.
 Statements by the President
 Ladies and gentlemen, on Saturday, as you know, an earthquake struck Central America once again, with tragic consequences.
 This is an area which has already been seriously affected on a number of occasions since the beginning of the twentieth century.
 The latest, provisional, figures for victims in El Salvador are already very high.
 There are 350 people dead, 1 200 people missing, the area is completely devastated and thousands of homes have been destroyed throughout the country.
 The European Union has already shown its solidarity by sending a rescue team to the area, whilst financial assistance from the Union and Member States has been, or is in the process of being, released and I am able to inform you that some groups in the European Parliament have requested that